In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



/Users/azizurrahman/miniconda3/envs/rag/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeConfigurationError: You haven't specified an Api-Key.

In [5]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Sarah Johnson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Johnson is an amazing professor! She is extremely knowledgeable and passionate about the subject matter. Her lectures are engaging and she is always willing to help students during office hours. Highly recommend this course.'},
 {'professor': 'Professor Michael Lee',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Professor Lee is a fair and reasonable instructor. His lectures can be dry at times, but he is very clear in explaining complex economic concepts. The workload is manageable, and he is accessible during office hours.'},
 {'professor': 'Dr. Emily Chen',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Dr. Chen is an exceptional professor. Her lectures are well-organized and she uses a variety of teaching methods to ensure students understand the material. She is also very responsive to student questions and concerns.'},
 {'professor': 'Professor David Gonzalez',
  'subjec

In [15]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [16]:
processed_data[0]

{'values': [0.0024006974,
  0.009312675,
  0.004926144,
  0.040430356,
  -0.009393907,
  -0.0064405417,
  0.0078563,
  0.03038079,
  -0.018834233,
  0.02222277,
  0.026829788,
  -0.0009863892,
  -0.025483657,
  0.007891114,
  0.00034614286,
  0.04114984,
  -0.030589672,
  -0.007421129,
  0.034372766,
  0.057001695,
  0.023731366,
  -0.023255577,
  0.011169408,
  -0.009480942,
  -0.052313447,
  -0.034071047,
  0.012231227,
  0.021828214,
  0.045768462,
  -0.018172773,
  0.0856418,
  0.011192617,
  -0.020714175,
  -0.029289959,
  -0.03920027,
  0.01821919,
  0.01892707,
  0.037041817,
  0.011384092,
  0.010658806,
  -0.0031970618,
  -0.023011882,
  -0.018683374,
  0.0098174745,
  -0.0032289743,
  -0.0027531865,
  0.0240679,
  -0.01687306,
  0.04305299,
  0.051292244,
  -0.036600843,
  -0.0041457363,
  0.030009443,
  -0.018195981,
  -0.032957006,
  -0.00026690532,
  0.023406437,
  0.030404,
  0.011523347,
  -0.03093781,
  0.053798832,
  -0.016629362,
  -0.02956847,
  -0.011384092,
  -0.00

In [17]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

{'upserted_count': 21}

In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}